In [ ]:
pip install openai==0.28

In [ ]:
import requests
from bs4 import BeautifulSoup
import openai
import re

def fetch_case_html(slug):
    """
    Fetches the HTML content of a case from the Casetext API using the provided slug.

    Args:
    slug (str): The Casetext case identifier.

    Returns:
    str: The HTML content of the case.
    """
    url = f"https://casetext.com/api/search-api/doc/{slug}/html"
    response = requests.get(url)
    response.raise_for_status()
    return response.text

def extract_relevant_sections(html_content):
    """
    Extracts relevant text sections from the HTML content that may indicate negative treatment of a case.

    Args:
    html_content (str): The HTML content of the case.

    Returns:
    list of str: A list of words indicating potential negative treatment.
    """
    soup = BeautifulSoup(html_content, 'html.parser')
    sections = []
    words = {
        'overrule', 'inconsistent', 'outdated law', 'overturn', 'abrogate',
        'limited', 'decline to follow', 'distinguished', 'abandon'
    }

    for section in soup.find_all(['p', 'div']):  # Adjusted to only fetch relevant text containing tags
        text = section.get_text(separator=' ')
        for word in words:
            if word in text.lower():
                sections.append(text)
                break  # If a word is found, no need to check other words

    return sections

def analyze_text_with_llm(text):
    """
    Analyzes the given text using OpenAI's GPT-3.5-turbo to identify negative treatments.

    Args:
    text (str): The text to be analyzed.

    Returns:
    str or None: The analysis result if a case name is mentioned- otherwise none.
    """
    openai.api_key =
    prompt = (
        f"Analyze the following legal text for any form of negative treatments, including those that limit, modify, or implicitly overrule precedents without explicitly stating so. "
        f"Identify the treated case, the nature of the treatment, and provide a detailed explanation of why it constitutes negative treatment. "
        f"If there is no negative treatment, explicitly state that:\n\n"
        f"{text}"
    )
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=300  # Increased max tokens to allow for clear and detailed responses
    )
    analysis = response.choices[0].message.content.strip()

    # Regular expression to detect case names (ex. "Johnson v. State")
    case_pattern = re.compile(r'\b[A-Z][a-z]+ v\. [A-Z][a-z]+\b')

    # To check if the analysis mentions a specific
    # previously was analysis included parties and/or evidential matters
    if case_pattern.search(analysis):
        return analysis
    else:
        return None

def extract_negative_treatments(slug):
    """
    Extracts negative treatments from a case identified by the provided slug.

    Args:
    slug (str): The Casetext case identifier.

    Returns:
    list of dict: A list of dictionaries containing section text and analysis results.
    """
    html_content = fetch_case_html(slug)
    sections = extract_relevant_sections(html_content)
    treatments = []
    for section in sections:
        analysis = analyze_text_with_llm(section)
        if analysis:
            treatments.append({
                'section_text': section,
                'analysis': analysis
            })
    if not treatments:
        treatments.append({
            'section_text': "No relevant sections found.",
            'analysis': "No negative treatment detected"
        })
    return treatments

def process_multiple_slugs(slugs):
    """
    Processes multiple case slugs to extract negative treatments from each one.

    Args:
    slugs (list of str): A list of Casetext case identifiers.

    Returns:
    dict: A dictionary with slugs as keys and the treatment analyses as values.
    """
    results = {}
    for slug in slugs:
        print(f"Processing slug: {slug}")
        treatments = extract_negative_treatments(slug)
        results[slug] = treatments
        for treatment in treatments:
            print("Section Text:", treatment['section_text'])
            print("Analysis:", treatment['analysis'])
            print()
    return results

# I did not receive results on last two cases, therefore, I read the cases to see if the findings were correct. They were correct in terms of In re: Lee, but there was a nuance in Tilden v. State that was not caught. I found that by manually reading the cases and then adjusted the prompt.
# def extract_text_from_html(html_content):
#     """
#     Extracts plain text from the given HTML content.
#
#     Args:
#     html_content (str): The HTML content of the case.
#
#     Returns:
#     str: The plain text extracted from the HTML.
#     """
#     soup = BeautifulSoup(html_content, 'html.parser')
#     text = soup.get_text(separator='\n')
#     return text

# def print_case_text(slug):
#     """
#     Fetches and prints the entire text of a case identified by the provided slug.
#
#     Args:
#     slug (str): The Casetext case identifier.
#     """
#     html_content = fetch_case_html(slug)
#     case_text = extract_text_from_html(html_content)
#     print(case_text)

# Fetch and print the entire text of "tilden-v-state"
if __name__ == "__main__":
    # Uncomment to print the entire case text
    # print_case_text('tilden-v-state')

    sample_text = "To the extent that State v. Trusty, is inconsistent with this holding, we overrule it."
    analysis = analyze_text_with_llm(sample_text)
    # print("Sample Analysis:", analysis)

    slugs = [
        'littlejohn-v-state-7',
        'beattie-v-beattie',
        'travelers-indem-co-v-lake',
        'tilden-v-state',
        'in-re-lee-342013'
    ]

    results = process_multiple_slugs(slugs)


Processing slug: littlejohn-v-state-7
Section Text:  The argument is that the only evidence offered by the State was the statement made by the deceased to the Robinsons that Littlejohn threw coal oil on her and threw a lighted match on her. This evidence is characterized as circumstantial which, under the rule, must be inconsistent with any hypothesis other than guilt.  Holland v. State,   9 Terry 559 ,  107 A.2d 920 . Thus, it is argued, the statement made by the deceased is as consistent with the view that the coal oil had been thrown and ignited accidentally. 
Analysis: In the provided legal text, the case of  Holland v. State, 9 Terry 559 , 107 A.2d 920  is referenced. The nature of the treatment can be considered negative as the case is implicitly overruled without being explicitly stated. This can be seen in the argument presented that the evidence offered by the State, which is similar to the case in  Holland v. State, 9 Terry 559 , 107 A.2d 920 , is not sufficient to establish 

In [ ]:
import unittest
from unittest.mock import patch, Mock


class TestCaseAnalysis(unittest.TestCase):

    @patch('requests.get')
    def test_fetch_case_html(self, mock_get):
        mock_get.return_value = Mock(ok=True)
        mock_get.return_value.text = '<html></html>'
        html_content = fetch_case_html('test-slug')
        self.assertEqual(html_content, '<html></html>')

    def test_extract_relevant_sections(self):
        html_content = '<div>overruled decision</div><div>another text</div>'
        sections = extract_relevant_sections(html_content)
        self.assertEqual(len(sections), 1)
        self.assertIn('overruled decision', sections[0])

    @patch('openai.ChatCompletion.create')
    def test_analyze_text_with_llm(self, mock_create):
        mock_create.return_value = Mock(choices=[Mock(message=Mock(content="Case: Johnson v. State\nNature of treatment: Request to overrule holding"))])
        analysis = analyze_text_with_llm('sample text')
        self.assertEqual(analysis, 'Case: Johnson v. State\nNature of treatment: Request to overrule holding')

        mock_create.return_value = Mock(choices=[Mock(message=Mock(content="No negative treatment"))])
        analysis = analyze_text_with_llm('sample text')
        self.assertIsNone(analysis)

    @patch('requests.get')
    @patch('openai.ChatCompletion.create')
    def test_extract_negative_treatments(self, mock_create, mock_get):
        slug = 'test-slug'
        html_content = '<div>overruled decision</div><div>another text</div>'
        mock_get.return_value = Mock(ok=True)
        mock_get.return_value.text = html_content
        mock_create.return_value = Mock(choices=[Mock(message=Mock(content="Case: Johnson v. State\nNature of treatment: Request to overrule holding"))])

        treatments = extract_negative_treatments(slug)
        self.assertEqual(len(treatments), 1)
        self.assertIn('overruled decision', treatments[0]['section_text'])
        self.assertEqual(treatments[0]['analysis'], 'Case: Johnson v. State\nNature of treatment: Request to overrule holding')

        # Test case where no negative treatment is detected
        mock_create.return_value = Mock(choices=[Mock(message=Mock(content="No negative treatment"))])
        treatments = extract_negative_treatments(slug)
        self.assertEqual(len(treatments), 1)
        self.assertEqual(treatments[0]['section_text'], "No relevant sections found.")
        self.assertEqual(treatments[0]['analysis'], "No negative treatment detected")

# Run the tests
unittest.main(argv=[''], verbosity=2, exit=False)


test_analyze_text_with_llm (__main__.TestCaseAnalysis) ... ok
test_extract_negative_treatments (__main__.TestCaseAnalysis) ... ok
test_extract_relevant_sections (__main__.TestCaseAnalysis) ... ok
test_fetch_case_html (__main__.TestCaseAnalysis) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.041s

OK
